In [1]:
import pandas as pd
import numpy as np
import os
import sys
from shapely.geometry import Polygon, Point
from tqdm import tqdm

### Read the data

In [2]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [3]:
def points_inside_polygon(closed_path):
    # IMPORTANT: these points should be ordered
    # Create a Shapely Polygon from the closed path
    polygon = Polygon(closed_path)

    # Find the bounding box of the polygon
    min_x, min_y, max_x, max_y = polygon.bounds

    # Generate all possible points within the bounding box
    possible_points = [(x, y) for x in range(int(min_x), int(max_x) + 1) for y in range(int(min_y), int(max_y) + 1)]

    # exclude points that are already in the closed path
    possible_points = [point for point in possible_points if point not in closed_path]

    # Filter points that are inside the polygon
    enclosed_points = list()
    for point in tqdm(possible_points):
        if polygon.contains(Point(point)):
            enclosed_points.append(point)

    return enclosed_points

In [4]:
working_file = file_content

working_file = [[x, int(y), z.replace('(', '').replace(')', '')] for x, y, z in [line.split(' ') for line in working_file]]

path = [[9999, 9999]]

for direction, n_steps, hex_color in working_file:
    if direction == 'U':
        for i in range(n_steps):
            path = path + [[path[-1][0] - 1, path[-1][1]]]
    elif direction == 'R':
        for i in range(n_steps):
            path = path + [[path[-1][0], path[-1][1] + 1]]
    elif direction == 'D':
        for i in range(n_steps):
            path = path + [[path[-1][0] + 1, path[-1][1]]]
    elif direction == 'L':
        for i in range(n_steps):
            path = path + [[path[-1][0], path[-1][1] - 1]]

min_y = min([x[0] for x in path])
min_x = min([x[1] for x in path])

# make 0, 0 the origin
path = [(x[0] - min_y, x[1] - min_x) for x in path]

enclosed_points = points_inside_polygon(path)

# join path and enclosed points
all_holes = path + enclosed_points

print(len(set(all_holes)))


100%|██████████| 111136/111136 [01:08<00:00, 1612.41it/s]

40131


### Part 2

In [5]:
def shoelace(tuples_list):
    position, area_a, area_b, total_distance = 0 + 0j, 0, 0, 0
    for direction, distance in tuples_list:
        match direction:
            case 0:
                new_position = position + (1j * distance)
            case 1:
                new_position = position + distance
            case 2:
                new_position = position - (1j * distance)
            case 3:
                new_position = position - distance
        area_a += position.real * new_position.imag
        area_b += position.imag * new_position.real
        total_distance += distance
        position = new_position
    return int((abs(area_a - area_b) / 2) + (total_distance / 2) + 1)

plan = [tuple(line.split()) for line in open('day18.txt').read().splitlines()]
print(shoelace([('RDLU'.index(direction), int(distance))
                for direction, distance, _ in plan]))
print(shoelace([(int(hex_string[-2]), int(hex_string[2:-2], base=16))
                for _, _, hex_string in plan]))

40131
104454050898331
